This code calibrates the stochastic HyMoLAP model following the the two-step method described in the paper.

2- The ranges of $\mu$, $\lambda$, and $\sigma$ parameters are defined by starting with large parameter ranges, which we manually reduced over time to achieve more accurate and robust results.

3- We only consider here the "Obs-Guided" scenario here. This can be adjusted to simulate the "7-Day Obs-Sim-Guided" scenario.







In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install kaleido==0.2.1

  Using cached kaleido-0.2.1-py2.py3-none-manylinux1_x86_64.whl.metadata (15 kB)
Using cached kaleido-0.2.1-py2.py3-none-manylinux1_x86_64.whl (79.9 MB)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import read_csv
import math
import random
from datetime import datetime, timedelta
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [ ]:
# Import data
data = pd.read_excel('/content/drive/MyDrive/Qdaily_1FG01_Yala_2014_2020.xlsx')

daily = data.iloc[:, [-4, -3, -2]]

#X(t)
def state_basin(MU, LANDA, q):
    n = len(q)
    X = np.zeros(n)
    X[0] = q[0]
    for i in range(1, n):
        if q[i] == 0:
            X[i] = X[i - 1] - (MU / LANDA) * X[i - 1]
        else:
            X[i] = X[i - 1] + (MU / LANDA) * q[i]
    return X

In [ ]:
# Performance metrics

def nash_sutcliffe_efficiency(observed, simulated):
    obs_mean = np.mean(observed)
    numerator = np.sum((observed - simulated) ** 2)
    denominator = np.sum((observed - obs_mean) ** 2)

    nse = 1 - (numerator / denominator)
    return nse

def calculate_rmse(observed, predicted):

    rmse = np.sqrt(np.mean((observed - predicted)**2))
    return rmse

Heuristic method for HyMoLAP

In [ ]:
#Calibration data
discharge = daily.iloc[:1461, 2].to_numpy()
prec = daily.iloc[:1461, 0].to_numpy()
pet= daily.iloc[:1461, 1].to_numpy()

#Compute the Effective precipitation
prec_eff = prec - pet
prec_eff[prec_eff < 0] = 0


x = discharge

# initialization
nashe = 0
S = np.zeros(len(x))
S[0] = x[0]
Qsim = np.zeros(len(x))

for pp in range(5000):
    mu = (1.2 - 0.95) * np.random.rand() + 0.95
    lambda_ = (23.5 - 15) * np.random.rand() + 15

    X = state_basin(mu, lambda_, prec_eff)

    for k in range(1, len(discharge)):
        if k in range(0, len(discharge), 1):   #This becomes range(0, len(discharge), 7) in the case of the "7-Day Obs-Sim-Guided" scenario
            S[k] = discharge[k - 1] - (mu/lambda_) * discharge[k-1]**(2*mu-1) + (1/lambda_) * X[k-1] * prec_eff[k-1]
        else:
            S[k] = S[k - 1] - (mu/lambda_) * S[k-1]**(2*mu-1) + (1/lambda_) * X[k-1] * prec_eff[k-1]

    # Nash-Sutcliffe efficiency calculation
    nas = nash_sutcliffe_efficiency(x, S)
    if nashe < nas:
        nashe = nas
        mu1 = mu
        lambda1 = lambda_
        Qsim = S.copy()


In [ ]:
rmse = calculate_rmse(discharge[1:], Qsim[1:])

In [ ]:
print('nse = ', nashe)
print('rmse = ', rmse)

nse =  0.8910181204365675
rmse =  7.41418565886466


In [ ]:
#Estimates of the HyMoLAP
print(mu1)
print(lambda1)

1.0220853264003489
21.319795879190185


Validation of HyMoLAP

In [ ]:
# Validation data
discharge1 = daily.iloc[1461:2191, 2].to_numpy()
prec1= daily.iloc[1461:2191, 0].to_numpy()
pet1 = daily.iloc[1461:2191, 1].to_numpy()

#Compute the Effective precipitation
prec_eff1 = prec1 - pet1
prec_eff1[prec_eff1 < 0] = 0

# Simulation
size = len(discharge1)

X = state_basin(mu1, lambda1, prec_eff1)

S = np.zeros(size)
S[0] = discharge1[0]

for k in range(1, len(discharge1)):
    if k in range(0, len(discharge1), 1):
        S[k] = discharge1[k - 1] - (mu1/lambda1) * discharge1[k-1]**(2*mu1-1) + (1/lambda1) * X[k-1] * prec_eff1[k-1]
    else:
        S[k] = S[k - 1] - (mu1/lambda1)* S[k-1]**(2*mu1-1) + (1/lambda1) * X[k-1] * prec_eff1[k-1]



In [ ]:
nse_value = nash_sutcliffe_efficiency(discharge1[1:], S[1:])
print("Nash-Sutcliffe Efficiency (NSE):", nse_value)

print('rmse=',calculate_rmse(discharge1[1:], S[1:]))

Nash-Sutcliffe Efficiency (NSE): 0.9232186017191306
rmse= 10.908809551064232


SDE MODEL

In [ ]:
# Compute the diffusion parameter

X = state_basin(mu1, lambda1, prec_eff)

num = 0
den = 0
for k in range(1, len(discharge)):
  num = num + (discharge[k] - discharge[k - 1] + (mu1/lambda1) * discharge[k-1]**(2*mu1-1) - (1/lambda1) * X[k-1] * prec_eff[k - 1])**2
  den = den + (discharge[k - 1])**2

sigma = np.sqrt(num/den)

In [ ]:
print(sigma)

0.17767390468174976


Calibration phase with 10,000 trajectories

In [ ]:
# Simulation with the SDE
size = len(discharge)
n_traj = 10000
QQ = np.zeros((size,n_traj))

X = state_basin(mu1, lambda1, prec_eff)

for i in range(n_traj):

   # Model
   S = np.zeros(size)
   S[0] = discharge[0]

   for k in range(1, len(discharge)):
      if k in range(0, len(discharge), 1):
          S[k] = (discharge[k - 1] - (mu1/lambda1) * discharge[k-1]**(2*mu1-1) + (1/lambda1) * X[k-1] * prec_eff[k-1] +
                     sigma * discharge[k - 1] * np.random.normal(loc=0, scale=np.sqrt(1)))
      else:
          S[k] = (S[k - 1] - (mu1/lambda1) * S[k-1]**(2*mu1-1) + (1/lambda1) * X[k-1] * prec_eff[k-1] +
                     sigma * S[k - 1] * np.random.normal(loc=0, scale=np.sqrt(1)))

   QQ[:, i] = S.copy()

mean_trajectory = np.mean(QQ, axis=1)


In [ ]:
nse_value = nash_sutcliffe_efficiency(discharge[1:], mean_trajectory[1:])
print("Nash-Sutcliffe Efficiency (NSE):", nse_value)

print('rmse=',calculate_rmse(discharge[1:], mean_trajectory[1:]))

Nash-Sutcliffe Efficiency (NSE): 0.8909151437709092
rmse= 7.409552005296676


In [ ]:
# Build 90% quantile interval
min_trajectory = np.percentile(QQ, 5, axis=1)
max_trajectory = np.percentile(QQ, 95, axis=1)

In [ ]:
import plotly.graph_objects as go

# Create a date range
time1 = np.datetime64('2014-01-02')
time2 = np.datetime64('2017-12-31')
time = np.arange(time1, time2 + np.timedelta64(1, 'D'), dtype='datetime64[D]')

# Create traces
upper_trace = go.Scatter(
    x=time,
    y=max_trajectory[1:],
    mode='lines',
    line=dict(color='rgba(169, 169, 169, 0.7)', width=2),
    showlegend=False
)

lower_trace = go.Scatter(
    x=time,
    y=min_trajectory[1:],
    mode='lines',
    line=dict(color='rgba(169, 169, 169, 0.7)', width=2),
    fill='tonexty',
    fillcolor='rgba(169, 169, 169, 0.7)',
    name='90% CI'  # Name for the legend
)

mode_trace = go.Scatter(
    x=time,
    y=mean_trajectory[1:],
    mode='lines',
    line=dict(color='red', width=2),
    name='Predicted'
)

real_trace = go.Scatter(
    x=time,
    y=discharge[1:],
    mode='lines',
    line=dict(color='blue', width=2),
    name='Observed'
)

# Create the figure
fig = go.Figure(data=[upper_trace, lower_trace, mode_trace, real_trace])

# Update layout
fig.update_layout(
    title=r'HyMoLAP Calibration',
    xaxis=dict(
        title='Time t',
        title_font=dict(size=25, color='black'),  # Darker and larger text
        tickfont=dict(size=18, color='black')    # Larger and darker labels
    ),
    yaxis=dict(
        title='Discharge Q [m³/s]',
        title_font=dict(size=22, color='black'), # Darker and larger text
        tickfont=dict(size=18, color='black')   # Larger and darker labels
    ),
    legend=dict(
        title='Legend',
        title_font=dict(size=20, color='black'), # Dark text
        font=dict(size=18, color='black')       # More readable legend
    ),
    template='plotly_white',
    width=800,  # Reduced width
    height=400   # Reduced height
)

# Save the figure as a JPEG image
#fig.write_image("sde_cal_90.jpeg",  scale=20)

#Download
#files.download("sde_cal_90.jpeg")

# If you need to display the figure in a Jupyter notebook or similar environment
fig.show()


In [ ]:
observed = discharge
lower_bound = min_trajectory
upper_bound = max_trajectory

# Calculate the "hits" and "misses" for the Hit Rate (HR)
hits = np.sum((observed >= lower_bound) & (observed <= upper_bound))
misses = len(observed) - hits
hit_rate = (hits / (hits + misses)) * 100  # HR formula in percentage

# Calculate the Mean Prediction Interval Width (MPIW)
mpiw = np.mean(upper_bound - lower_bound)

print(f"Hit Rate (HR) in %: {hit_rate:}")
print(f"Mean Prediction Interval Width (MPIW): {mpiw}")


Hit Rate (HR) in %: 91.03353867214237
Mean Prediction Interval Width (MPIW): 17.53859443653268


Validation with 10,000 trajectories

In [ ]:
# Simulation with the SDE
size = len(discharge1)
n_traj = 10000
QQ = np.zeros((size,n_traj))

X = state_basin(mu1, lambda1, prec_eff1)

for i in range(n_traj):

   # Model
   S = np.zeros(size)
   S[0] = discharge1[0]

   for k in range(1, len(discharge1)):
      if k in range(0, len(discharge1), 1):
          S[k] = (discharge1[k - 1] - (mu1/lambda1) * discharge1[k-1]**(2*mu1-1) + (1/lambda1) * X[k-1] * prec_eff1[k-1] +
                               sigma * discharge1[k - 1] * np.random.normal(loc=0, scale=np.sqrt(1)))
      else:
          S[k] = (S[k - 1] - (mu1/lambda1)* S[k-1]**(2*mu1-1) + (1/lambda1) * X[k-1] * prec_eff1[k-1] +
                               sigma * S[k - 1] * np.random.normal(loc=0, scale=np.sqrt(1)))

   QQ[:, i] = S.copy()

mean_trajectory1 = np.mean(QQ, axis=1)


In [ ]:
nse_value = nash_sutcliffe_efficiency(discharge1[1:], mean_trajectory1[1:])
print("Nash-Sutcliffe Efficiency (NSE):", nse_value)

print('rmse=',calculate_rmse(discharge1[1:], mean_trajectory1[1:]))

Nash-Sutcliffe Efficiency (NSE): 0.9232514966121013
rmse= 10.90647250997661


In [ ]:
# Build 90% quantile interval
min_trajectory1 = np.percentile(QQ, 5, axis=1)
max_trajectory1 = np.percentile(QQ, 95, axis=1)

In [ ]:
import plotly.graph_objects as go

# Create a date range from '01-01-2000' to '12-31-2005'
time1 = np.datetime64('2018-01-02')
time2 = np.datetime64('2019-12-31')
time = np.arange(time1, time2 + np.timedelta64(1, 'D'), dtype='datetime64[D]')

# Trace for the upper bound without a name in the legend
upper_trace = go.Scatter(
    x=time,
    y=max_trajectory1[1:],
    mode='lines',
    line=dict(color='rgba(169, 169, 169, 0.7)', width=2),
    showlegend=False  # Does not display this trace in the legend
)

# Trace for the lower bound with a name in the legend
lower_trace = go.Scatter(
    x=time,
    y=min_trajectory1[1:],
    mode='lines',
    line=dict(color='rgba(169, 169, 169, 0.7)', width=2),
    fill='tonexty',
    fillcolor='rgba(169, 169, 169, 0.7)',  # Uncertainty area in gray
    name='90% QI'  # Name for the legend
)

# Trace for the mode
mode_trace = go.Scatter(
    x=time,
    y=mean_trajectory1[1:],
    mode='lines',
    line=dict(color='red', width=2),
    name = 'Predicted'
)

# Trace for the observed values
real_trace = go.Scatter(
    x=time,
    y=discharge1[1:],
    mode='lines',
    line=dict(color='blue', width=2),
    name='Observed'
)

# Add the traces to the figure
data = [upper_trace, lower_trace, mode_trace, real_trace]
fig = go.Figure(data=data)

# Update the title with LaTeX text
fig.update_layout(
    title=r'90% Quantile Interval (QI) with SDE',
    xaxis=dict(
        title='Time t',
        title_font=dict(size=25, color='black'),  # Darker and larger text
        tickfont=dict(size=18, color='black')    # Larger and darker tick labels
    ),
    yaxis=dict(
        title='Discharge Q [m³/s]',
        title_font=dict(size=22, color='black'), # Darker and larger text
        tickfont=dict(size=18, color='black')   # Larger and darker tick labels
    ),
    legend=dict(
        title='Legend',
        title_font=dict(size=20, color='black'), # Dark text
        font=dict(size=18, color='black')       # More readable legend
    ),
    template='plotly_white',
    width=800,  # Reduced width
    height=400   # Reduced height
)

# Save the figure as a JPEG image
fig.write_image("sde_val_100000_90.jpeg",  scale=22)

# Download
#files.download("sde_val_100000_90.jpeg")

# If you need to display the figure in a Jupyter notebook or similar environment
fig.show()


In [ ]:
observed = discharge1
lower_bound = min_trajectory1
upper_bound = max_trajectory1

# Calculate the "hits" and "misses" for the Hit Rate (HR)
hits = np.sum((observed >= lower_bound) & (observed <= upper_bound))
misses = len(observed) - hits
hit_rate = (hits / (hits + misses)) * 100  # HR formula in percentage

# Calculate the Mean Prediction Interval Width (MPIW)
mpiw = np.mean(upper_bound - lower_bound)

print(f"Hit Rate (HR) in %: {hit_rate:}")
print(f"Mean Prediction Interval Width (MPIW): {mpiw}")


Hit Rate (HR) in %: 92.73972602739727
Mean Prediction Interval Width (MPIW): 26.590138818894893


Calibration phase with 100,000 trajectories

In [ ]:
# Simulation with the SDE
size = len(discharge)
n_traj = 100000
QQ = np.zeros((size,n_traj))

X = state_basin(mu1, lambda1, prec_eff)

for i in range(n_traj):

   # Model
   S = np.zeros(size)
   S[0] = discharge[0]

   for k in range(1, len(discharge)):
      if k in range(0, len(discharge), 1):
          S[k] = (discharge[k - 1] - (mu1/lambda1) * discharge[k-1]**(2*mu1-1) + (1/lambda1) * X[k-1] * prec_eff[k-1] +
                     sigma * discharge[k - 1] * np.random.normal(loc=0, scale=np.sqrt(1)))
      else:
          S[k] = (S[k - 1] - (mu1/lambda1) * S[k-1]**(2*mu1-1) + (1/lambda1) * X[k-1] * prec_eff[k-1] +
                     sigma * S[k - 1] * np.random.normal(loc=0, scale=np.sqrt(1)))

   QQ[:, i] = S.copy()

mean_trajectory = np.mean(QQ, axis=1)


In [ ]:
nse_value = nash_sutcliffe_efficiency(discharge[1:], mean_trajectory[1:])
print("Nash-Sutcliffe Efficiency (NSE):", nse_value)

print('rmse=',calculate_rmse(discharge[1:], mean_trajectory[1:]))

Nash-Sutcliffe Efficiency (NSE): 0.8909050336303178
rmse= 7.409895361283167


In [ ]:
# Build 90% quantile interval
min_trajectory = np.percentile(QQ, 5, axis=1)
max_trajectory = np.percentile(QQ, 95, axis=1)

In [ ]:
import plotly.graph_objects as go

# Create a date range
time1 = np.datetime64('2014-01-02')
time2 = np.datetime64('2017-12-31')
time = np.arange(time1, time2 + np.timedelta64(1, 'D'), dtype='datetime64[D]')

# Create traces
upper_trace = go.Scatter(
    x=time,
    y=max_trajectory[1:],
    mode='lines',
    line=dict(color='rgba(169, 169, 169, 0.7)', width=2),
    showlegend=False
)

lower_trace = go.Scatter(
    x=time,
    y=min_trajectory[1:],
    mode='lines',
    line=dict(color='rgba(169, 169, 169, 0.7)', width=2),
    fill='tonexty',
    fillcolor='rgba(169, 169, 169, 0.7)',
    name='90% CI'  # Name for the legend
)

mode_trace = go.Scatter(
    x=time,
    y=mean_trajectory[1:],
    mode='lines',
    line=dict(color='red', width=2),
    name='Predicted'
)

real_trace = go.Scatter(
    x=time,
    y=discharge[1:],
    mode='lines',
    line=dict(color='blue', width=2),
    name='Observed'
)

# Create the figure
fig = go.Figure(data=[upper_trace, lower_trace, mode_trace, real_trace])

# Update layout
fig.update_layout(
    title=r'HyMoLAP Calibration',
    xaxis=dict(
        title='Time t',
        title_font=dict(size=25, color='black'),  # Darker and larger text
        tickfont=dict(size=18, color='black')    # Larger and darker labels
    ),
    yaxis=dict(
        title='Discharge Q [m³/s]',
        title_font=dict(size=22, color='black'), # Darker and larger text
        tickfont=dict(size=18, color='black')   # Larger and darker labels
    ),
    legend=dict(
        title='Legend',
        title_font=dict(size=20, color='black'), # Dark text
        font=dict(size=18, color='black')       # More readable legend
    ),
    template='plotly_white',
    width=800,  # Reduced width
    height=400   # Reduced height
)

# Save the figure as a JPEG image
#fig.write_image("sde_cal_90.jpeg",  scale=20)

#Download
#files.download("sde_cal_90.jpeg")

# If you need to display the figure in a Jupyter notebook or similar environment
fig.show()


In [ ]:
observed = discharge
lower_bound = min_trajectory
upper_bound = max_trajectory

# Calculate the "hits" and "misses" for the Hit Rate (HR)
hits = np.sum((observed >= lower_bound) & (observed <= upper_bound))
misses = len(observed) - hits
hit_rate = (hits / (hits + misses)) * 100  # HR formula in percentage

# Calculate the Mean Prediction Interval Width (MPIW)
mpiw = np.mean(upper_bound - lower_bound)

print(f"Hit Rate (HR) in %: {hit_rate:}")
print(f"Mean Prediction Interval Width (MPIW): {mpiw}")


Hit Rate (HR) in %: 91.10198494182067
Mean Prediction Interval Width (MPIW): 17.54254070333956


Validation with 100,000 trajectories

In [ ]:

# Simulation with the SDE
size = len(discharge1)
n_traj = 100000
QQ = np.zeros((size,n_traj))

X = state_basin(mu1, lambda1, prec_eff1)

for i in range(n_traj):

   # Model
   S = np.zeros(size)
   S[0] = discharge1[0]

   for k in range(1, len(discharge1)):
      if k in range(0, len(discharge1), 1):
          S[k] = (discharge1[k - 1] - (mu1/lambda1) * discharge1[k-1]**(2*mu1-1) + (1/lambda1) * X[k-1] * prec_eff1[k-1] +
                    sigma * discharge1[k - 1] * np.random.normal(loc=0, scale=np.sqrt(1)))
      else:
          S[k] = (S[k - 1] - (mu1/lambda1)* S[k-1]**(2*mu1-1) + (1/lambda1) * X[k-1] * prec_eff1[k-1] +
                    sigma * S[k - 1] * np.random.normal(loc=0, scale=np.sqrt(1)))

   QQ[:, i] = S.copy()

mean_trajectory1 = np.mean(QQ, axis=1)


In [ ]:
nse_value = nash_sutcliffe_efficiency(discharge1[1:], mean_trajectory1[1:])
print("Nash-Sutcliffe Efficiency (NSE):", nse_value)

print('rmse=',calculate_rmse(discharge1[1:], mean_trajectory1[1:]))

Nash-Sutcliffe Efficiency (NSE): 0.9231534935719623
rmse= 10.913433730159783


In [ ]:
# Build 90% quantile interval
min_trajectory1 = np.percentile(QQ, 5, axis=1)
max_trajectory1 = np.percentile(QQ, 95, axis=1)

In [ ]:
import plotly.graph_objects as go

# Create a date range from '01-01-2000' to '12-31-2005'
time1 = np.datetime64('2018-01-02')
time2 = np.datetime64('2019-12-31')
time = np.arange(time1, time2 + np.timedelta64(1, 'D'), dtype='datetime64[D]')


# Trace for the upper bound without a name in the legend
upper_trace = go.Scatter(
    x=time,
    y=max_trajectory1[1:],
    mode='lines',
    line=dict(color='rgba(169, 169, 169, 0.7)', width=2),
    showlegend=False  # Does not display this trace in the legend
)

# Trace for the lower bound with a name in the legend
lower_trace = go.Scatter(
    x=time,
    y=min_trajectory1[1:],
    mode='lines',
    line=dict(color='rgba(169, 169, 169, 0.7)', width=2),
    fill='tonexty',
    fillcolor='rgba(169, 169, 169, 0.7)',  # Uncertainty zone in gray
    name='90% QI'  # Name for the legend
)

# Trace for the mean
mode_trace = go.Scatter(
    x=time,
    y=mean_trajectory1[1:],
    mode='lines',
    line=dict(color='red', width=2),
    name = 'Predicted'
)

# Trace for the observed values
real_trace = go.Scatter(
    x=time,
    y=discharge1[1:],
    mode='lines',
    line=dict(color='blue', width=2),
    name='Observed'
)

# Add the traces to the figure
data = [upper_trace, lower_trace, mode_trace, real_trace]
fig = go.Figure(data=data)

# Update the title with LaTeX text
fig.update_layout(
    title=r'90% Quantile Interval (QI) with SDE',
    xaxis=dict(
        title='Time t',
        title_font=dict(size=25, color='black'),  # Darker and larger text
        tickfont=dict(size=18, color='black')    # Larger and darker tick labels
    ),
    yaxis=dict(
        title='Discharge Q [m³/s]',
        title_font=dict(size=22, color='black'), # Darker and larger text
        tickfont=dict(size=18, color='black')   # Larger and darker tick labels
    ),
    legend=dict(
        title='Legend',
        title_font=dict(size=20, color='black'), # Dark text
        font=dict(size=18, color='black')       # More readable legend
    ),
    template='plotly_white',
    width=800,  # Reduced width
    height=400   # Reduced height
)

# Save the figure as a JPEG image
fig.write_image("sde_val_100000_90.jpeg",  scale=22)

# Download
#files.download("sde_val_100000_90.jpeg")

# If you need to display the figure in a Jupyter notebook or similar environment
fig.show()


In [ ]:
observed = discharge1
lower_bound = min_trajectory1
upper_bound = max_trajectory1

# Calculate the "hits" and "misses" for the Hit Rate (HR)
hits = np.sum((observed >= lower_bound) & (observed <= upper_bound))
misses = len(observed) - hits
hit_rate = (hits / (hits + misses)) * 100  # HR formula in percentage

# Calculate the Mean Prediction Interval Width (MPIW)
mpiw = np.mean(upper_bound - lower_bound)

print(f"Hit Rate (HR) in %: {hit_rate:}")
print(f"Mean Prediction Interval Width (MPIW): {mpiw}")


Hit Rate (HR) in %: 92.73972602739727
Mean Prediction Interval Width (MPIW): 26.595920954725806
